<a href="https://colab.research.google.com/github/Nicordaro/Project_MLDL/blob/master/Project_MLDL_con_classe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Politecnico di Torino**

**01TXFSM - Machine learning and Deep learning**

**Incremental Learning in Image Classification**

**Cordaro Nicolò - s272145**

**Di Nepi Marco - s277959**

**Falletta Alberto - s277971**


In [0]:
# !pip3 install 'torch==1.4.0'
# !pip3 install 'torchvision==0.5.0'
# !pip3 install 'Pillow-SIMD'
# !pip3 install 'tqdm'

**Imports**

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn


import torchvision
from torchvision import transforms
from torchvision.models import resnet18

from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
from google.colab import drive

**Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 10

SEED = 42 # Used for the pseudorandom shuffle of the split

BATCH_SIZE = 256  
LR = 0.01           
MOMENTUM = 0.9       
WEIGHT_DECAY = 5e-5  

NUM_EPOCHS = 15  
STEP_SIZE = 20       
GAMMA = 0.1

LOG_FREQUENCY = 10

**Pre-processing**

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      
                                      transforms.CenterCrop(224),         
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
                                      ])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                 
])

**Prepare Dataset**

CIFAR100 has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another.

Each of the downloaded files is a Python "pickled" object produced with cPickle.

In [6]:
# Clone github repository with data
# if os.path.isdir('./Project_MLDL'):
#   !rm -rf Project_MLDL
if not os.path.isdir('./CIFAR100_tError'):
  !git clone https://github.com/Nicordaro/Project_MLDL


fatal: destination path 'Project_MLDL' already exists and is not an empty directory.


data -- a 50000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.

labels -- a list of 50000 numbers in the range 0-99. The number at index i indicates the label of the ith image in the array data.

**Create dataset**

In [0]:
import PIL.Image
import os
import os.path
import numpy as np
import pickle
import torch

from torchvision.datasets import VisionDataset


# from .utils import check_integrity, download_and_extract_archive
from torchvision.datasets import CIFAR100

def pil_loader(f):
    # open file
    img = PIL.Image.open(f)
    return img.convert('RGB')


class CIFAR100_tError(CIFAR100):

  def __init__(self, root, train=True, transform=None,download=False, lbls=[]):
    flag = train
    self.prova = CIFAR100(root, train=train, download=download)
    self.transform = transform
    self.data = []
    self.labels = []
    for element in self.prova:
      image, label = element
      if label in lbls:
        self.data.append(image)
        self.labels.append(label)

  def __len__(self):
        '''
        The __len__ method returns the length of the dataset
        It is mandatory, as this is used by several other components
        '''
        length = len(self.data)
        return length
      
          
  def __getitem__(self, index):
      '''
      __getitem__ should access an element through its index
      Args:
          index (int): Index
      Returns:
          tuple: (sample, target) where target is class_index of the target class.
      '''
      
      image = self.data[index]
      label = self.labels[index]
      
      if self.transform is not None:
          image = self.transform(image)
      
      return image, label
    
  def increment(self, newlbls):
    for element in self.prova:
      image, label = element
      if label in newlbls:
        self.data.append(image)
        self.labels.append(label)
      


In [0]:
#from Project_MLDL.CIFAR100_tError import CIFAR100_tError
import numpy as np
import random

DATA_DIR = './CIFAR100'

lbls = [i for i in range(0,100)]
random.seed(SEED)
random.shuffle(lbls)

added_labels=[]
new_labels=[]

def make_data_labels(lbls):
  new_labels=[]
  for el in lbls[:10]:
    added_labels.append(el)
    new_labels.append(el)
  lbls = lbls[10:]

  return added_labels, lbls, new_labels

def increment_dataset(new_labels):
  CIFAR100_tError.increment(new_labels)

In [22]:
train_dataset = CIFAR100_tError(DATA_DIR, train=True, transform=train_transform, download=True)
test_dataset = CIFAR100_tError(DATA_DIR, train=False, transform=eval_transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


**Dataloaders**

In [0]:
added_labels, lbls, new_labels = make_data_labels(lbls)
train_dataset.increment(new_labels) #load 10 classes
test_dataset.increment(new_labels)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

**Prepare Network**

In [0]:
net = resnet18(pretrained=True)
net.fc = nn.Linear(512, NUM_CLASSES)

**Prepare Training**

In [0]:
# Loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Parameters to optimize:
parameters_to_optimize = net.parameters()

# Optimizer
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
# Scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Training**


In [26]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
  
  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)
    
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  # Step the scheduler
  scheduler.step()

Starting epoch 1/15, LR = [0.01]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0, Loss 2.527050733566284
Step 10, Loss 1.96030592918396
Starting epoch 2/15, LR = [0.01]
Step 20, Loss 1.9993001222610474
Step 30, Loss 1.962334156036377
Starting epoch 3/15, LR = [0.01]
Step 40, Loss 1.8523123264312744
Step 50, Loss 1.9852763414382935
Starting epoch 4/15, LR = [0.01]
Step 60, Loss 1.985669493675232
Step 70, Loss 1.8978134393692017
Starting epoch 5/15, LR = [0.01]
Step 80, Loss 1.8702114820480347
Step 90, Loss 1.5893949270248413
Starting epoch 6/15, LR = [0.01]
Step 100, Loss 1.8949542045593262
Step 110, Loss 1.8960829973220825
Starting epoch 7/15, LR = [0.01]
Step 120, Loss 1.9786536693572998
Step 130, Loss 1.8909844160079956
Starting epoch 8/15, LR = [0.01]
Step 140, Loss 1.8786568641662598
Step 150, Loss 1.928051471710205
Starting epoch 9/15, LR = [0.01]
Step 160, Loss 1.7295058965682983
Step 170, Loss 1.8996551036834717
Starting epoch 10/15, LR = [0.01]
Step 180, Loss 1.9703319072723389
Starting epoch 11/15, LR = [0.01]
Step 190, Loss 1.908834457397461
Step 2

**Test**

In [28]:
#test phase
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

100%|██████████| 19/19 [00:09<00:00,  2.00it/s]

Test Accuracy: 0.0


In [0]:
for i in range(0,10):
  #create new train dataset
  #increment new labels
  #create new dataloader
  #increment test dataset
  #change last layer resnet
  #train
  #test